In [ ]:
import os
import sys
import numpy as np 
import pandas as pd
import re
import random
import pickle
import torch
import torch.optim as optim
from torchtext.data.metrics import bleu_score
from transformers import T5ForConditionalGeneration, T5Tokenizer

In [2]:
!pip install -U sentence-transformers
!pip install -U rouge

     |████████████████████████████████| 64 kB 1.1 MB/s eta 0:00:011
  Created wheel for sentence-transformers: filename=sentence_transformers-0.4.1.2-py3-none-any.whl size=103066 sha256=fe13a8089423aa94b7cb1f2ad4b5daefbd4b0d4faa113456e8a00669199859af
  Stored in directory: /root/.cache/pip/wheels/1c/a2/db/5d9e7c4aa8dbd82718c202fb1bc7118c2d3bf0925af92943f2
Successfully built sentence-transformers


In [6]:
r= [1,3, 23, 34, 63, 77, 106, 114, 115, 129, 134, 145, 160, 163, 165, 170, 175, 176, 183, 185, 188, 198, 245, 249, 256,260, 294, 332, 341, 355, 360, 363, 386, 392, 394, 398]

In [4]:

from rouge import Rouge
file=open('test-data/article_test_all_trans_eng.pkl','rb')


test=pickle.load(file)
file2=open('train-data/article_train_all_trans.pkl','rb')


train=pickle.load(file2)



In [13]:
data_train=[]
data_val=[]
for i,j in zip(train['Text'],train['Headline']):
    data_train.append([i,j])

for i,j in zip(test['Text'],test['Headline']):
    
    data_val.append([i,j]) 
    

In [15]:
def preprocess_article(articles):
    for article in articles:
        article = re.sub(r"http\S+", "", article)
        article = re.sub(r"www.\S+", "", article)
        article = re.sub(r"<\S+", "", article)
        article = re.sub('\n+', " ",article)
        article = article.strip()
       
    return articles


In [16]:
for i in range(5):
    print(data_train[i])

['Sign in Welcome! Log into your account your username', '#Paytm यूजर्स के लिए बड़ी खुशखबरी']
['Subscription E-Commerce Market Stay up-to-date with Subscription E-Commerce Market research offered by HTF MI.  Check how key trends and emerging drivers are shaping this industry growth.  A New business research report released by HTF MI with title COVID-19 Outbreak-Global Subscription E-Commerce Market Study Forecast till 2026 .  This COVID-19 Outbreak-Global Subscription E-Commerce market report brings data for the estimated year 2020 and forecasted till 2026 in terms of both, value (US$ MN) and volume (MT).  The report also consists of forecast factors, macroeconomic factors, and a market outlook of the COVID-19 Outbreak- Subscription E-Commerce market.  The study is conducted by applying both top-down and bottom-up approaches and further iterative methods used to validate and size market estimation and trends of the COVID-19 Outbreak-Global Subscription E-Commerce market.  Additionally 

In [17]:

for i in range(len(data_train)):
        data_train[i][0]= preprocess_article(data_train[i][0].lstrip().rstrip().lower())
        data_train[i][1]= preprocess_article(data_train[i][1].lstrip().rstrip().lower())
for i in range(len(data_val)):
        data_val[i][0]= preprocess_article(data_val[i][0].lstrip().rstrip().lower())
        data_val[i][1]= preprocess_article(data_val[i][1].lstrip().rstrip().lower())

#data_train=data[:3000]
#data_val=data[3000:]


model=T5ForConditionalGeneration.from_pretrained('t5-base').cuda()
tokenizer=T5Tokenizer.from_pretrained('t5-base')
rouge1=Rouge()

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

In [20]:
def rouge(sentence,ref):
    s=''
    flag=0
    for char in sentence:
        if char=='<':
            flag=1
        elif char=='>':
            flag=0
        elif flag==0:
            s+=char

    gen=re.sub('\s+',' ',s).rstrip().lstrip()
#     print(gen)
    if gen=='':
        return 0,0,0
    rg=rouge1.get_scores(gen,ref)
    r1,r2,rl=rg[0]["rouge-1"]['f'], \
    rg[0]["rouge-2"]['f'],rg[0]["rouge-l"]['f']
    
    return r1,r2,rl

def generate_batch(data):
    output=random.sample(data,4)

    inp,label=[],[]
    for dat in output:
            inp.append(dat[0])
            label.append(dat[1])
            
    return inp,label
def val(dev_data):
    with torch.no_grad():
        r1_,r2_,rl_=0,0,0
        candidate_corpus,references_corpus=[],[]

        for line in dev_data:
            inp,label=[line[0]],[line[1]]
            input=tokenizer.prepare_seq2seq_batch(src_texts=inp,
                                                  tgt_texts=label, padding=True, return_tensors='pt')

            output=model.generate(input_ids=input['input_ids'].cuda(),
                                  num_beams=5, early_stopping=True, max_length=20)
            out=tokenizer.batch_decode(output)
            torch.cuda.empty_cache()
            out[0] = re.sub(r"<\S+", "", out[0])
            #print(label[0])
            #print(out[0])
            #print("--------------------------------------------------------------------------------------------------------------")
            candidate_corpus.append(tokenizer.tokenize(out[0]))
            references_corpus.append([tokenizer.tokenize(label[0])])
            
            r1,r2,rl=rouge(out[0],line[1])
            del output
            del out
            del input
            del inp
            del label
            r1_+=r1
            r2_+=r2
            rl_+=rl

        r1_/=(len(dev_data)*0.01)
        r2_/=(len(dev_data)*0.01)
        rl_/=(len(dev_data)*0.01)
        bleu=0
        bleu=100*bleu_score(candidate_corpus, references_corpus)

        return r1_,r2_,rl_,bleu


# Train

In [21]:


optimizer=optim.AdamW(model.parameters(),lr=0.00001)

scalar=0
val_score=0
for i in range(3000):
        model.train()
        inp,label=generate_batch(data_train)
        input=tokenizer.prepare_seq2seq_batch(src_texts=inp, tgt_texts=label, padding=True, return_tensors='pt',max_length=600,truncation=True)
        outputs=model(input_ids=input['input_ids'].cuda(),labels=input['labels'].cuda())
        loss=outputs[0]
        
        scalar+=loss.item()
        torch.cuda.empty_cache()
        del outputs
        del input
        del inp
        del label
        if(i+1)%50==0:
                print('iteration={}, training loss={}'.format(i+1,scalar/(4*50)))
                scalar=0
        if(i+1)%3000==0:
                r1,r2,rl,bleu=val(data_x)
                print('validation BLEU={}, validation R1={}, validation R2={}, validation RL={}'.format(bleu,r1,r2,rl))
                

        loss.backward()
        optimizer.step()

iteration=50, training loss=1.1582945156097413
iteration=100, training loss=0.6915342581272125
iteration=150, training loss=0.6186654216051102
iteration=200, training loss=0.5549868321418763
iteration=250, training loss=0.5175634422898292
iteration=300, training loss=0.4748349702358246
iteration=350, training loss=0.4163611927628517
iteration=400, training loss=0.450492101162672
iteration=450, training loss=0.44888476610183714
iteration=500, training loss=0.4688331997394562
iteration=550, training loss=0.4223625707626343
iteration=600, training loss=0.4333072492480278
iteration=650, training loss=0.4417973732948303
iteration=700, training loss=0.4046163882315159
iteration=750, training loss=0.37463323086500167
iteration=800, training loss=0.3895107523351908
iteration=850, training loss=0.3823143169283867
iteration=900, training loss=0.36099899366497995
iteration=950, training loss=0.37028553038835527
iteration=1000, training loss=0.3615856957435608
iteration=1050, training loss=0.33915

# Test Data

In [22]:
r1,r2,rl,bleu=val(data_val)
print('validation BLEU={}, validation R1={}, validation R2={}, validation RL={}'.format(bleu,r1,r2,rl))              

validation BLEU=28.904716014317906, validation R1=39.72569546875942, validation R2=26.8504839602334, validation RL=38.68267632367739


In [ ]:
torch.save(model.state_dict(),'t5_mod.pth')

In [ ]:
torch.save(optimizer.state_dict(),'optt5_.pth')